<a href="https://colab.research.google.com/github/jnardosp/foldwalker/blob/main/hp_model_mcmc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Parte 1:** Convertir una cadena de aminoácidos a un lattice 3D

## Hydrophobic - Polar Lattice Model MonteCarlo sampling

Let's perform an initial MCMC sampling on a HP model to later do approximate counting of proteins.

Aminoacids & H (Hydrophobic) - P (Polar/Neutral) Classification:
| Aminoacid     | Letter | H or P |
|---------------|--------|--------|
| Alanine       | A      | H      |
| Arginine      | R      | P      |
| Asparagine    | N      | P      |
| Aspartate     | D      | P      |
| Cysteine      | C      | H      |
| Glutamine     | Q      | P      |
| Glutamate     | E      | P      |
| Glycine       | G      | P      |
| Histidine     | H      | P      |
| Isoleucine    | I      | H      |
| Leucine       | L      | H      |
| Lysine        | K      | P      |
| Methionine    | M      | H      |
| Phenylalanine | F      | H      |
| Proline       | P      | P      |
| Serine        | S      | P      |
| Threonine     | T      | P      |
| Tryptophan    | W      | H      |
| Tyrosine      | Y      | H      |
| Valine        | V      | H      |

[Aminoacids & HP](https://www.alfa-chemistry.com/resources/hydrophobicity-index-table-of-common-amino-acids.html)


In [2]:
import plotly.graph_objects as go
import numpy as np

In [6]:
# HP classification mapping
HP_MAP = {
    'A': 'H', 'C': 'H', 'I': 'H', 'L': 'H', 'M': 'H',
    'F': 'H', 'W': 'H', 'Y': 'H', 'V': 'H',
    'R': 'P', 'N': 'P', 'D': 'P', 'Q': 'P', 'E': 'P',
    'G': 'P', 'H': 'P', 'K': 'P', 'P': 'P', 'S': 'P', 'T': 'P'
}

def generate_3d_saw(sequence):
    """Generate 3D self-avoiding walk coordinates for the sequence."""
    # Directions in 3D space
    directions = {
        0: (1, 0, 0),   # right
        1: (-1, 0, 0),  # left
        2: (0, 1, 0),   # forward
        3: (0, -1, 0),  # backward
        4: (0, 0, 1),   # up
        5: (0, 0, -1)   # down
    }

    # Initialize
    coords = [(0, 0, 0)]
    visited = {(0, 0, 0)}
    hp_types = [HP_MAP.get(sequence[0], 'P')]

    # Generate self-avoiding walk
    for aa in sequence[1:]:
        current = coords[-1]
        possible_moves = []

        # Check all possible directions
        for dir_idx in range(6):
            dx, dy, dz = directions[dir_idx]
            new_pos = (current[0] + dx, current[1] + dy, current[2] + dz)

            # Avoid collisions
            if new_pos not in visited:
                possible_moves.append(new_pos)

        # If stuck, backtrack or extend in minimal direction
        if not possible_moves:
            # Simple fallback: try to find any unoccupied neighbor
            # by checking extended neighborhood
            found = False
            for dir_idx in range(6):
                dx, dy, dz = directions[dir_idx]
                new_pos = (current[0] + dx, current[1] + dy, current[2] + dz)
                if new_pos not in visited:
                    coords.append(new_pos)
                    visited.add(new_pos)
                    hp_types.append(HP_MAP.get(aa, 'P'))
                    found = True
                    break

            if not found:
                # Add position with minimal movement
                new_pos = (current[0] + 1, current[1], current[2])
                coords.append(new_pos)
                visited.add(new_pos)
                hp_types.append(HP_MAP.get(aa, 'P'))
        else:
            # Choose first available move
            new_pos = possible_moves[0]
            coords.append(new_pos)
            visited.add(new_pos)
            hp_types.append(HP_MAP.get(aa, 'P'))

    return coords, hp_types

def plot_hp_lattice_3d(sequence):
    """Plot 3D HP lattice using Plotly."""
    # Generate coordinates
    coords, hp_types = generate_3d_saw(sequence)

    # Separate H and P residues
    h_coords = [coords[i] for i, hp in enumerate(hp_types) if hp == 'H']
    p_coords = [coords[i] for i, hp in enumerate(hp_types) if hp == 'P']

    # Create figure
    fig = go.Figure()

    # Add hydrophobic residues (H) as red spheres
    if h_coords:
        h_x, h_y, h_z = zip(*h_coords)
        fig.add_trace(go.Scatter3d(
            x=h_x, y=h_y, z=h_z,
            mode='markers',
            marker=dict(size=12, color='red'),
            name='Hydrophobic (H)',
            text=[f'Residue {i+1}: {sequence[i]} (H)'
                  for i, hp in enumerate(hp_types) if hp == 'H']
        ))

    # Add polar residues (P) as blue spheres
    if p_coords:
        p_x, p_y, p_z = zip(*p_coords)
        fig.add_trace(go.Scatter3d(
            x=p_x, y=p_y, z=p_z,
            mode='markers',
            marker=dict(size=10, color='blue', opacity=0.8),
            name='Polar (P)',
            text=[f'Residue {i+1}: {sequence[i]} (P)'
                  for i, hp in enumerate(hp_types) if hp == 'P']
        ))

    # Add connections between residues
    x_coords, y_coords, z_coords = zip(*coords)
    fig.add_trace(go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='lines',
        line=dict(color='gray', width=4),
        name='Protein Backbone'
    ))

    # Update layout
    fig.update_layout(
        title=f'3D HP Lattice Model: {sequence}',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='cube',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
        ),
        margin=dict(l=0, r=0, b=0, t=40),
        showlegend=True
    )

    fig.show()

    # Print statistics
    print(f"Sequence: {sequence}")
    print(f"Length: {len(sequence)} residues")
    print(f"H residues: {hp_types.count('H')}")
    print(f"P residues: {hp_types.count('P')}")
    print(f"Unique positions: {len(set(coords))}")

# Example usage
if __name__ == "__main__":
    # Test sequence - you can replace with your own
    test_sequence = "ACDEFGHIKLMNPQRSTVWY"

    coords, hp_types = generate_3d_saw(test_sequence)
    print("Coords: ", coords)
    print("HP_types: ", hp_types)

    # Generate and plot
    plot_hp_lattice_3d(test_sequence)

Coords:  [(0, 0, 0), (1, 0, 0), (2, 0, 0), (3, 0, 0), (4, 0, 0), (5, 0, 0), (6, 0, 0), (7, 0, 0), (8, 0, 0), (9, 0, 0), (10, 0, 0), (11, 0, 0), (12, 0, 0), (13, 0, 0), (14, 0, 0), (15, 0, 0), (16, 0, 0), (17, 0, 0), (18, 0, 0), (19, 0, 0)]
HP_types:  ['H', 'H', 'P', 'P', 'H', 'P', 'P', 'H', 'P', 'H', 'H', 'P', 'P', 'P', 'P', 'P', 'P', 'H', 'H', 'H']


Sequence: ACDEFGHIKLMNPQRSTVWY
Length: 20 residues
H residues: 9
P residues: 11
Unique positions: 20


## **Parte 2:** calcular la función de energía de un lattice 3D

In [11]:
import numpy as np
from itertools import combinations
from typing import Sequence, Union, Callable, Tuple, List, Dict, Optional

def _default_U(a: str, b: str) -> float:
    """Interacción por defecto: -1 si ambos H, 0 en otro caso (modelo HP)."""
    return -1.0 if a.upper() == 'H' and b.upper() == 'H' else 0.0

def energy_hp(coords: Union[np.ndarray, Sequence[Sequence[float]]],
              seq: Union[str, Sequence[str]],
              U: Optional[Union[Callable[[str,str], float], Dict[str, Dict[str, float]]]] = None,
              contact_distance: float = 1.0,
              tol: float = 1e-8,
              metric: str = 'euclidean',
              return_contacts: bool = False
              ) -> Union[float, Tuple[float, List[Tuple[int,int,float]]]]:
    """
    Calcula la energía E_HP(C) = sum_{i=0}^{N-3} sum_{j=i+2}^{N-1} U(h_i,h_j) * C_ij,
    donde C_ij = 1 si i y j son vecinos más cercanos en la retícula (distancia == contact_distance),
    y U(h_i,h_j) es la potencial de interacción (por defecto: -1 si ambos 'H', 0 en otro caso).

    Parámetros:
    - coords: array-like (N,3) con coordenadas de cada residuo (enteras o floats).
    - seq: secuencia de tipos (ej. "HHPHP" o ['H','P',...]).
    - U: función U(a,b)->float o dict-of-dicts U[a][b]; si None usa el modelo HP por defecto.
    - contact_distance: distancia que define vecinos (por defecto 1.0).
    - tol: tolerancia numérica para comparar distancias.
    - metric: 'euclidean' (por defecto) o 'manhattan'.
    - return_contacts: si True, devuelve además lista de (i,j,contribución).

    Devuelve:
    - energía (float), o (energía, contactos) si return_contacts=True.
    """
    coords = np.asarray(coords, dtype=float)
    if coords.ndim != 2 or coords.shape[1] != 3:
        raise ValueError("coords debe ser array de forma (N,3)")
    N = coords.shape[0]
    if isinstance(seq, str):
        seq_list = list(seq.upper())
    else:
        seq_list = [str(s).upper() for s in seq]
    if len(seq_list) != N:
        raise ValueError("La longitud de seq debe coincidir con el número de coordenadas")

    # normalizar U a una función callable
    if U is None:
        Ucall = _default_U
    elif callable(U):
        Ucall = U
    elif isinstance(U, dict):
        def Ucall(a,b):
            a = a.upper(); b = b.upper()
            # intentar dict-of-dicts con simetría si es necesario
            if a in U and b in U[a]:
                return float(U[a][b])
            if b in U and a in U[b]:
                return float(U[b][a])
            # intentar clave tuple (a,b)
            if (a,b) in U:
                return float(U[(a,b)])
            if (b,a) in U:
                return float(U[(b,a)])
            # fallback
            return 0.0
    else:
        raise ValueError("U debe ser None, callable o dict-of-dicts")

    E = 0.0
    contacts: List[Tuple[int,int,float]] = []

    # Iterar sobre pares i<j con |i-j| >= 2 (i=0..N-3, j=i+2..N-1)
    for i, j in combinations(range(N), 2):
        if abs(i - j) < 2:
            continue  # saltar pares consecutivos
        diff = coords[i] - coords[j]
        if metric == 'euclidean':
            d = np.linalg.norm(diff)
            is_contact = abs(d - contact_distance) <= tol
        elif metric == 'manhattan':
            d = np.sum(np.abs(diff))
            is_contact = abs(d - contact_distance) <= tol
        else:
            raise ValueError("metric debe ser 'euclidean' o 'manhattan'")

        if is_contact:
            contrib = float(Ucall(seq_list[i], seq_list[j]))
            if contrib != 0.0:
                E += contrib
            # guardamos el contacto y la contribución (incluso si 0) para debug
            contacts.append((i, j, contrib))

    if return_contacts:
        return E, contacts
    return E

In [24]:
# Tests simples
coords1 = [(0,0,0),(1,0,0),(2,0,0),(3,0,0)]
seq1 = "HHHH"
print("Lineal (sin contactos):", energy_hp(coords1, seq1))  # 0

coords2 = [(0,0,0),(1,0,0),(1,1,0),(0,1,0)]
seq2 = "HHHH"
print("i=0,j=3 contacto:", energy_hp(coords2, seq2))  # -1

coords3 = [(0,0,0),(1,0,0),(2,0,0),(2,1,0),(1,1,0),(0,1,0)]
seq3 = "HHHHHH"
print("i=0,j=5 contacto y i=1,j=4 contacto:", energy_hp(coords3, seq3))  # -2

coords3 = [(0,0,0),(1,0,0),(2,0,0),(2,1,0),(1,1,0),(0,1,0)]
seq3 = "HHHHHP"
print("i=1,j=4 contacto:", energy_hp(coords3, seq3))  # -1

Lineal (sin contactos): 0.0
i=0,j=3 contacto: -1.0
i=0,j=5 contacto y i=1,j=4 contacto: -2.0
i=1,j=4 contacto: -1.0


In [25]:
# Para debugging: devolver índices de los contactos y contribución de cada uno
coords3 = [(0,0,0),(1,0,0),(2,0,0),(2,1,0),(1,1,0),(0,1,0)]
seq3 = "HHHHHH"
print("i=0,j=5 contacto y i=1,j=4 contacto:", energy_hp(coords3, seq3, return_contacts=True))  # -2

i=0,j=5 contacto y i=1,j=4 contacto: (-2.0, [(0, 5, -1.0), (1, 4, -1.0)])
